# The encryption algorithm

In [1]:
import sys
from struct import pack, unpack

def F(w):
    return ((w * 31337) ^ (w * 1337 >> 16)) % 2**32

def encrypt(block):
    a, b, c, d = unpack("<4I", block)
    for rno in xrange(32):
        a, b, c, d = b ^ F(a | F(c ^ F(d)) ^ F(a | c) ^ d), c ^ F(a ^ F(d) ^ (a | d)), d ^ F(a | F(a) ^ a), a ^ 31337
        a, b, c, d = c ^ F(d | F(b ^ F(a)) ^ F(d | b) ^ a), b ^ F(d ^ F(a) ^ (d | a)), a ^ F(d | F(d) ^ d), d ^ 1337
    return pack("<4I", a, b, c, d)

# Reverse engineering the encryption algorithm
- First it's noticed that the algorithm works on blocks of 16 bytes
- Each block is divided into four 4-byte ints (a,b,c,d)
- 32 Rounds are performed on these ints
- Each round performs two operations on these 4 ints
- We will need to figure out a way to reverse these functions

In [2]:
def reverse_second_op(a1, b1, c1, d1):
    # Variables should be decoded in the given sequence d0, a0, b0, c0
    d0 = d1 ^ 1337
        
    fd0 = F(d0 | F(d0) ^ d0)
    a0 = c1 ^ fd0

    fad0 = F(d0 ^ F(a0) ^ (d0 | a0))
    b0 = b1 ^ fad0

    fabd0 = F(d0 | F(b0 ^ F(a0)) ^ F(d0 | b0) ^ a0)

    c0 = a1 ^ fabd0
    
    return a0, b0, c0, d0

In [3]:
def reverse_first_op(a1, b1, c1, d1):
    a0 = d1 ^ 31337
    
    fa0 = F(a0 | F(a0) ^ a0)
    d0 = c1 ^ fa0
    
    fad0 = F(a0 ^ F(d0) ^ (a0 | d0))
    c0 = b1 ^ fad0
    
    facd0 = F(a0 | F(c0 ^ F(d0)) ^ F(a0 | c0) ^ d0)
    b0 = a1 ^ facd0
    
    return  a0, b0, c0, d0

In [4]:
def decrypt(block):
    a, b, c, d = unpack("<4I", block)
    for rno in range(32):
        # Second operation is reversed first
        a, b, c, d = reverse_second_op(a, b, c, d)
        # First operation is reversed
        a, b, c, d = reverse_first_op(a, b, c, d)
    
    return pack("<4I", a, b, c, d)

# How to encrypt a new message?

In [5]:
def encrypt_file(file_name):
    pt = open(file_name).read()
    while len(pt) % 16: pt += "#"

    ct = "".join(encrypt(pt[i:i+16]) for i in xrange(0, len(pt), 16))
    open(file_name + ".enc", "w").write(ct)

# How to decrypt an encrypted message?

In [6]:
def decrypt_file(file_name):
    pt = open(file_name, 'rb').read()
    # No need to add any padding

    # Decode the bytes stream using .decode() before joining the list
    ct = ''.join([decrypt(pt[i:i+16]).decode() for i in range(0, len(pt), 16)])
    print(ct)

In [7]:
decrypt_file('assets/genfei/flag.enc')

FLAG{G3N3R4L123D_F31573L_EZ!}###
